# Ciarlet-Raviart method for Kirchhoff-Love plates
The Ciarlet-Raviart method for Kirchhoff-Love plates is based on the following idea. Assume that the material tensor $\mathcal{C}$ is the identity (or a multiple of it). Then the strong form of the Kirchhoff-Love plate is equivalent to a bi-Laplacian problem

\begin{align*}
\mathrm{div}\mathrm{div}(\nabla^2 w) = \Delta^2 w = f.
\end{align*}

We define the scalar $\sigma=\Delta w $ (the trace of the full bending moment tensor $\sigma$ from the HHJ method) as an additional unknown. Then, for clamped boundaries $w\in H_0^2(\Omega)$, we obtain the following saddle-point problem: Find $(w,\sigma)\in H^1_0(\Omega)\times H^1(\Omega)$ such that

\begin{align*}
&\int_{\Omega} \sigma\,\delta\sigma\,dx + \int_{\Omega}\nabla\delta\sigma\cdot\nabla w\,dx& &= 0 \qquad \forall \delta\sigma\in H^1(\Omega),\\
& \int_{\Omega}\nabla\sigma\cdot\nabla\delta w\,dx& &= -\int_{\Omega}f\,\delta w\,dx \qquad \forall \delta w\in H_0^1(\Omega).
\end{align*}
It looks simpler than the HHJ method, as $\sigma$ is now a scalar in $H^1$. Let's see how good it performs.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [ ]:
def SolveHHJ(order, mesh, draw=False, clamped=".*", free="", simplysupp=""):
    V = HDivDiv(mesh, order=order, dirichlet=free+"|"+simplysupp)
    Q = H1(mesh, order=order+1, dirichlet=clamped+"|"+simplysupp)
    X = V*Q
    (sigma, w), (tau, v) = X.TnT()

    n = specialcf.normal(2)

    def tang(u): return u-(u*n)*n

    a = BilinearForm(X, symmetric=True)
    a += (InnerProduct(sigma, tau) + div(sigma)*grad(v) \
          + div(tau)*grad(w) - 1e-8*w*v )*dx \
          + (-(sigma*n) * tang(grad(v)) - (tau*n)*tang(grad(w)))*dx(element_boundary=True)
    a.Assemble()

    f = LinearForm(X)
    f += 100*v*dx
    f.Assemble()

    gfsol = GridFunction(X)
    gfsol.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * f.vec

    gfsigma, gfw = gfsol.components
    if draw:
        Draw(gfw, mesh, deformation=True)
        
    return gfw, gfsigma

def SolveCR(order, mesh, draw=False, clamped=".*", free="", simplysupp=""):
    V1 = H1(mesh, order=order, dirichlet=clamped+"|"+simplysupp)
    V2 = H1(mesh, order=order,dirichlet=free+"|"+simplysupp)
    V = V1*V2
    (w,sigma),(v,tau) = V.TnT()
    
    a = BilinearForm(V, symmetric=True)
    a += (sigma*tau + Grad(tau)*Grad(w)+Grad(sigma)*Grad(v) -1e-8*w*v)*dx
    a.Assemble()
    
    f = LinearForm(V)
    f += 100*v*dx
    f.Assemble()
    
    gfsol = GridFunction(V)
    gfw,gfsigma = gfsol.components
    
    gfsol.vec.data = a.mat.Inverse(V.FreeDofs(), inverse="sparsecholesky")*f.vec
    
    if draw:
        Draw(gfw, mesh, deformation=True)
        
    return gfw, gfsigma

Let's start with fully clamped boundaries.

In [ ]:
gfw1, gfsigma1 = SolveHHJ(order=2, mesh=mesh, draw=True, clamped=".*")
gfw2, gfsigma2 = SolveCR (order=3, mesh=mesh, draw=True, clamped=".*")

print("diff = ", sqrt(Integrate( (gfw1-gfw2)**2,mesh)))

As we see, both methods deliver good results. What about simply supported boundaries?

In [ ]:
gfw1, gfsigma1 = SolveHHJ(order=2, mesh=mesh, draw=True, clamped="left|right", free="", simplysupp="top|bottom")
gfw2, gfsigma2 = SolveCR (order=3, mesh=mesh, draw=True, clamped="left|right", free="", simplysupp="top|bottom")

print("diff = ", sqrt(Integrate( (gfw1-gfw2)**2,mesh)))

It works too. But, what about free boundaries?

In [ ]:
gfw1, gfsigma1 = SolveHHJ(order=2, mesh=mesh, draw=True, clamped="left|right", free="top|bottom", simplysupp="")
gfw2, gfsigma2 = SolveCR (order=3, mesh=mesh, draw=True, clamped="left|right", free="top|bottom", simplysupp="")

print("diff = ", sqrt(Integrate( (gfw1-gfw2)**2,mesh)))

We observe that the Ciarlet-Raviart method yields an entirely wrong solution, whereas the HHJ method performs well. It fails no matter how we try to incorporate the free boundary condition. This shows that the stress $\sigma$ must be discretized by a matrix; a scalar is not enough to account for all possible boundary conditions. However, if you only consider plates with Dirichlet and/or simply supported boundary conditions, the Ciarlet-Raviart method performs rather well.